# Tensorflow2.0-beta1

Start: Professional

1. import Tensorflow

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

2. Load MNIST dataset 

In [3]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Add dimension of channel
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

Shuffle datasets and make batch

In [4]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

Make `tf.keras` model using 'Model subclassing API'

In [5]:
class MyModel(Model) :
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = Conv2D(32, 3, activation='relu')
        self.flatten = Flatten()
        self.d1 = Dense(128, activation = 'relu')
        self.d2 = Dense(10, activation='softmax')
        
    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)
    
model = MyModel()

Select optimizer and loss function

In [6]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

Choose way to evaluate loss and accuracy

In [7]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name = 'train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name = 'test_accuracy')

Train with `tf.GradientTape`

In [9]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape :
        predictions = model(images)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    train_loss(loss)
    train_accuracy(labels, predictions)

Test Model

In [13]:
@tf.function
def test_step(images, labels):
    predictions = model(images)
    t_loss = loss_object(labels, predictions)
    
    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [14]:
EPOCHS = 5

for epoch in range(EPOCHS):
    for images, labels in train_ds:
        train_step(images, labels)
        
    for test_images, test_labels in test_ds:
        test_step(test_images, test_labels)
        
    template = '에포크: {}, 손실: {}, 정확도:{}, 테스트 손실:{}, 테스트 정확도: {}'
    print(template.format(epoch+1,
                         train_loss.result(),
                         train_accuracy.result()*100,
                         test_loss.result(),
                         test_accuracy.result()*100))

에포크: 1, 손실: 0.08725116401910782, 정확도:97.37317657470703, 테스트 손실:0.06035347655415535, 테스트 정확도: 98.05500030517578
에포크: 2, 손실: 0.06485065817832947, 정확도:98.04865264892578, 테스트 손실:0.06220611557364464, 테스트 정확도: 98.05999755859375
에포크: 3, 손실: 0.05176838859915733, 정확도:98.4360580444336, 테스트 손실:0.0658986046910286, 테스트 정확도: 98.07250213623047
에포크: 4, 손실: 0.043059248477220535, 정확도:98.69571685791016, 테스트 손실:0.06621494144201279, 테스트 정확도: 98.0999984741211
에포크: 5, 손실: 0.03713538870215416, 정확도:98.86839294433594, 테스트 손실:0.06725962460041046, 테스트 정확도: 98.1500015258789
